# An initial foray into computer algebra with SymPy

This is a first go at using a computer algebra system ([SymPy](https://www.sympy.org/en/index.html)) to explore the formal steady state of a [Stock-Flow Consistent model](https://en.wikipedia.org/wiki/Stock-Flow_consistent_model). For this, I use a simplified five-equation version of Godley and Lavoie's (2007) Model *SIM*. For more information, see my previous posts [here](http://www.christhoung.com/2015/12/08/sim-graph/) and [here](http://www.christhoung.com/2019/07/27/fsic-update/).

## Model

Assume a closed economy with no investment such that national income, $Y$, in time $t$ consists of household final consumption expenditure, $C$, and government expenditure, $G$:

\begin{equation} \tag{1}
Y_t = C_t + G_t
\end{equation}

Government expenditure $\left( G \right)$ is exogenous. Consumption $\left( C \right)$ depends on current disposable income, $YD$, and accumulated wealth, $H$ (from the previous period):

\begin{equation} \tag{2}
C_t = \alpha_1 \cdot YD_t + \alpha_2 \cdot H_{t-1}
\end{equation}

$\alpha_1$ and $\alpha_2$ are the marginal propensities to consume out of disposable income $\left( YD \right)$ and wealth $\left( H \right)$, respectively.

Disposable income $\left( YD \right)$ is national income $\left( Y \right)$ minus taxes, $T$, which are levied at a fixed proportion, $\theta$:

\begin{equation} \tag{3}
YD_t = Y_t - T_t
\end{equation}

\begin{equation} \tag{4}
T_t = \theta \cdot Y_t
\end{equation}

Households accumulate savings from the difference between their disposable income $\left( YD \right)$ and their expenditure $\left( C \right)$:

\begin{equation} \tag{5}
H_t = H_{t-1} + YD_t - C_t
\end{equation}

## Numerical solution

We can use FSIC to define and then solve the model numerically (see my [earlier post](http://www.christhoung.com/2018/07/08/fsic-gl2007-pc/) for details of the syntax):

In [1]:
import fsic

# Define the model's economic logic in a string
script = '''
Y = C + G
C = {alpha_1} * YD + {alpha_2} * H[-1]
YD = Y - T
T = {theta} * Y
H = H[-1] + YD - C
'''

# Translate into a set of symbols and a model class definition in one go
SIM = fsic.build_model(fsic.parse_model(script))

Simulate the model over a long enough period for it to achieve its *stationary state* (at which the values of the variables in levels are constant):

In [2]:
model = SIM(range(1, 100 + 1))  # Solve from Period 1 to 100

# Set propensities to consume
model.alpha_1 = 0.6
model.alpha_2 = 0.4

# Fiscal policy
model.G[1:] = 20   # Government spending
model.theta = 0.2  # Rate of income tax

model.solve()

We can then reproduce the main elements of Table 3.4 from Godley and Lavoie (2007) ('The impact of $20 of government expenditures, with perfect foresight').

While I've made no changes to FSIC for this post, I have added a new module to accompany the core library: [fsictools](https://github.com/ChrisThoung/website/blob/master/code/2019-12-03_sympy_sim/fsictools.py). This provides the [`to_dataframe()`](https://github.com/ChrisThoung/website/blob/master/code/2019-12-03_sympy_sim/fsictools.py#L35) function from an [earlier post](http://www.christhoung.com/2018/07/08/fsic-gl2007-pc/).

In [3]:
from fsictools import to_dataframe

# Convert the contents of the model to a `pandas` DataFrame
results = to_dataframe(model)

# Extract selected variables of interest, adding a new column for the change in
# wealth
extract = results[['G', 'Y', 'T', 'YD', 'C', 'H']].copy()
extract['D(H)'] = extract['H'].diff()

# Transpose, and select periods, to reproduce the main elements of Table 3.4
table_3_4 = extract.T
table_3_4[[1, 2, 3, 100]].round(1)

,1,2,3,100
G,0.0,20.0,20.0,20.0
Y,0.0,38.5,47.9,100.0
T,0.0,7.7,9.6,20.0
YD,0.0,30.8,38.3,80.0
C,0.0,18.5,27.9,80.0
H,0.0,12.3,22.7,80.0
D(H),NaN,12.3,10.4,0.0


By the one-hundredth period, the model has achieved a stationary state at which the values of the variables in levels are constant. The period-on-period change in household wealth is zero.

## Analytical solution

Model *SIM* is small and thus amenable to more formal algebraic analysis of its properties. The remainder of this post replicates key results from Section 3.5 of Godley and Lavoie (2007) using [SymPy](https://www.sympy.org/en/index.html). I'm new to [computer algebra systems](https://en.wikipedia.org/wiki/Computer_algebra_system) so this is both a first go and a note of my early learnings.

Define the system of equations in SymPy. Below, I use `1` as a suffix to indicate the one-period lag on wealth $\left( H \right)$.

In [4]:
from sympy import Symbol, Eq        # Term and equation objects
from sympy import symbols, sympify  # Helpers to generate SymPy objects
from sympy import factor, linsolve  # Expression manipulation and system solution

# Define symbols for the endogenous variables
Y, C, YD, T, H = symbols('Y, C, YD, T, H')

# Define the individual equations with left-hand and right-hand
# side expressions
output            = Eq(lhs=Y,  rhs=sympify('C + G'))
consumption       = Eq(lhs=C,  rhs=sympify('alpha_1 * YD + alpha_2 * H1'))
disposable_income = Eq(lhs=YD, rhs=sympify('Y - T'))
taxes             = Eq(lhs=T,  rhs=sympify('theta * Y'))
wealth            = Eq(lhs=H,  rhs=sympify('H1 + YD - C'))

# Assemble the system of equations as a dictionary, keyed by LHS
# variable (symbol)
system = {
    Y: output,
    C: consumption,
    YD: disposable_income,
    T: taxes,
    H: wealth,
}

In [5]:
# Display the system
for v in system.values():
    display(v)

Eq(Y, C + G)

Eq(C, H1*alpha_2 + YD*alpha_1)

Eq(YD, -T + Y)

Eq(T, Y*theta)

Eq(H, -C + H1 + YD)

At the model's stationary state, the values of the variables are constant in levels. As in the above numerical solution, the change in wealth should be zero:

\begin{equation}
\Delta H_t = H_t - H_{t-1} = 0
\end{equation}

\begin{equation}
H_t = H_{t-1} = H_{t-2} = H_{t-3} = \dots
\end{equation}

We can impose this on the system by replacing instances of `H1` (the one-period lag of wealth) with `H` (the contemporaneous value of wealth):

In [6]:
stationary_state = {
    k: v.subs(Symbol('H1'), Symbol('H'))
    for k, v in system.items()
}

In [7]:
# Display the system
for v in stationary_state.values():
    display(v)

Eq(Y, C + G)

Eq(C, H*alpha_2 + YD*alpha_1)

Eq(YD, -T + Y)

Eq(T, Y*theta)

Eq(H, -C + H + YD)

We can solve for the stationary state with [`linsolve()`](https://docs.sympy.org/latest/modules/solvers/solveset.html#sympy.solvers.solveset.linsolve) and inspect the values:

In [8]:
values = list(linsolve(list(stationary_state.values()),    # System of equations
                       list(stationary_state.keys())))[0]  # Unknowns to solve for
solution = dict(zip(stationary_state.keys(), values))

In [9]:
# Display the solution at its stationary state
for k, v in solution.items():
    display(Eq(k, factor(v)))

Eq(Y, G/theta)

Eq(C, -G*(theta - 1)/theta)

Eq(YD, -G*(theta - 1)/theta)

Eq(T, G)

Eq(H, G*(alpha_1 - 1)*(theta - 1)/(alpha_2*theta))

As in Equation 3.15 of Godley and Lavoie (2007), the stationary state for national income (the first result, $Y$) is:

\begin{equation}
Y^{\star} = \frac{G}{\theta}
\end{equation}

$\frac{G}{\theta}$ represents the *fiscal stance*: the ratio of government expenditure to its income share (also as in Godley and Cripps, 1983).

The expressions for household final consumption expenditure $\left( C \right)$ and disposable income $\left( YD \right)$ are the same. The two are equal. This is consistent with (but, here, also follows from) there being no change in wealth at the stationary state. With slight rearrangement:

\begin{equation}
C^{\star} = YD^{\star} = G \cdot \frac{1 - \theta}{\theta}
\end{equation}

At the stationary state, tax revenues $\left( T \right)$ match government expenditures $\left( G \right)$ such that government debt (the mirror image of household wealth) is constant.

Finally, the level of household wealth (or, conversely, government debt) is, again with some rearrangement:

\begin{equation}
H^{\star} = \frac{1 - \alpha_1}{\alpha_2} \cdot G \cdot \frac{1 - \theta}{\theta} = \alpha_3 \cdot G \cdot \frac{1 - \theta}{\theta} = \alpha_3 \cdot YD^{\star}
\end{equation}

\begin{equation}
\alpha_3 = \frac{1 - \alpha_1}{\alpha_2}
\end{equation}

See [here](https://github.com/ChrisThoung/website/tree/master/code/2019-12-03_sympy_sim) for this post as a Jupyter notebook along with supporting Python code.

## References

Godley, W., Cripps, F. (1983)
*Macroeconomics*,
Oxford University Press

Godley, W., Lavoie, M. (2007)
*Monetary economics: An integrated approach to
credit, money, income, production and wealth*,
Palgrave Macmillan